# Task 3: Regression for $Li_2CO_3$

In [31]:
import warnings 
warnings.filterwarnings('ignore')
target = "LiOH"
data_path = {
    "LiOH": "../Data/Lithium Hydroxide (wind database).csv", # Date, Price
    "Li2CO3": "../Data/Lithium Carbonate (wind database).csv", # Date, Price
    "Future": "../Data/Lithium futures price.csv" # Date,Open,High,Low,Price,Change,Volume
}
target_df = transform_data(data_path[target])

         Date  LiOH 56.5%
0  2005-01-04       27000
1  2005-01-05       27000
2  2005-01-06       27000
3  2005-01-07       27000
4  2005-01-10       27000


Please enter the column names, separated by a comma:  Date, Price


In [41]:
from time import time
from pathlib import Path
from tqdm import tqdm 

import yfinance as yf

import numpy as np
from numpy.linalg import LinAlgError
import pandas as pd
import requests
import io

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.tree import  DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV

import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VAR
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
import statsmodels.tsa.arima.model as tsa
from scipy.stats import probplot, moment
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

from dataloader import YahooDownloader

upstream_companies = pd.read_csv("../Data/Upstream_companies.csv")

In [71]:
def transform_data(file_path, sampling_period = 'W-FRI'): 
    
    '''
    This function transform the price into log return and output the weekly log return and price in a dataframe.
    '''

    outlier_cutoff = 0.01
    # read the data
    price = pd.read_csv(file_path)

    # print or inspect the DataFrame before renaming the columns
    print(price.head())

    # ask for the column names 
    column_names = input("Please enter the column names, separated by a comma: ").split(',')

    # strip any leading/trailing whitespace from the column names
    column_names = [name.strip() for name in column_names]

    # rename the columns 
    price.columns = column_names

    # transfer the date to date format 
    price['Date'] = pd.to_datetime(price['Date'])
    
    if (price['Price'].apply(type) == str).any():
        price['Price'] = price['Price'].str.replace(',', '', regex=True).astype(float)
    price.set_index('Date', inplace=True)

    # Resample to get the last business day's 'Close' price for each week.
    # 'W-FRI' denotes weeks ending on Friday.
    weekly_price = price['Price'].resample(sampling_period).last()

    # make the series a dataframe:
    weekly_price = pd.DataFrame(weekly_price)

    # Calculate log returns
    weekly_price['Log_Return'] = np.log(weekly_price["Price"]).diff().\
    pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))

    # The first entry will be NaN because there's no previous data to calculate a return from.
    # So, we remove the first entry
    weekly_price = weekly_price.dropna()

    return weekly_price

def plot_correlogram(x, lags=None, title=None):
    '''
    Plot the series, QQ plot, ACF, PACF
    '''
    lags = min(10, int(len(x)/5)) if lags is None else lags
    with sns.axes_style('whitegrid'):
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
        x.plot(ax=axes[0][0], title='Series(residuals)')
        x.rolling(21).mean().plot(ax=axes[0][0], c='k', lw=1)
        q_p = np.max(q_stat(acf(x, nlags=lags), len(x))[1])
        stats = f'Q-Stat: {np.max(q_p):>8.2f}\nADF: {adfuller(x)[1]:>11.2f}'
        axes[0][0].text(x=.02, y=.85, s=stats, transform=axes[0][0].transAxes)
        probplot(x, plot=axes[0][1])
        mean, var, skew, kurtosis = moment(x, moment=[1, 2, 3, 4])
        s = f'Mean: {mean:>12.2f}\nSD: {np.sqrt(var):>16.2f}\nSkew: {skew:12.2f}\nKurtosis:{kurtosis:9.2f}'
        
        axes[0][1].text(x=.02, y=.75, s=s, transform=axes[0][1].transAxes)
        
        plot_acf(x=x, lags=lags, ax=axes[1][0])
        plot_pacf(x, lags=lags, ax=axes[1][1])
        axes[1][0].set_xlabel('Lag')
        axes[1][1].set_xlabel('Lag')
        fig.suptitle(title, fontsize=14)
        sns.despine()
        fig.tight_layout()
        fig.subplots_adjust(top=.9)
        
def ARIMA_Model_Fit(series, p,d,q, title):
    '''
    Fit the ARIMA model and output the residual of the model 
    '''
    plot_correlogram(series, lags = 50, title = "Before Modeling: " + title)
    model_price = tsa.ARIMA(series,order=(p,d,q)).fit()
    print(model_price.summary())
    plot_correlogram(model_price.resid, lags = 50, title = "After Modeling: "+ title)
    return model_price.resid

def get_stocks(start_date, end_date, ticker_list):
    tickers = [ticker.split('.')[0] if ticker.endswith('US') else ticker for ticker in ticker_list]
    downloader = YahooDownloader(start_date = start_date, end_date = end_date, ticker_list = tickers)
    data = downloader.fetch_data()
    return data

In [8]:
ticker_list = upstream_companies.Stock
stocks = get_stocks(start_date = '2017-08-09', end_date = '2023-08-07', ticker_list = upstream_companies.Stock)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
stocks

,date,open,high,low,close,volume,tic,day
0,2017-08-09,11.84,12.13,11.70,11.90,34822277,002176.SZ,2
1,2017-08-09,29.67,30.79,29.41,29.23,46293439,002460.SZ,2
2,2017-08-09,48.01,49.47,47.18,46.34,24713128,002466.SZ,2
3,2017-08-09,13.80,14.47,13.61,13.80,80564320,002497.SZ,2
4,2017-08-09,3.20,3.21,3.18,3.19,705032,AKE.AX,2
...,...,...,...,...,...,...,...,...
35071,2023-08-04,50.00,50.34,49.40,50.34,9700,SGML.V,4
35072,2023-08-04,6.00,6.12,5.98,6.02,52600,SLI.V,4
35073,2023-08-04,70.70,72.94,70.50,71.05,1028300,SQM,4
35074,2023-08-04,0.15,0.15,0.14,0.14,22174671,SYA.AX,4


In [6]:
stocks_close = stocks[['date', 'tic', 'close']].copy()
pivoted_stocks = stocks_close.pivot(index='date', columns='tic', values='close')
pivoted_stocks_clean = pivoted_stocks.dropna()

In [7]:
pivoted_stocks_clean.index = pd.to_datetime(pivoted_stocks_clean.index)
pivoted_stocks_clean_friday = pivoted_stocks_clean.resample('W-FRI').last()
log_return_stocks = np.log(pivoted_stocks_clean_friday).diff().dropna()
log_return_stocks

tic,002176.SZ,002460.SZ,002466.SZ,002497.SZ,AKE.AX,ALB,ALL.L,AMLI,CRE.V,ERA.PA,...,LPI.AX,LTH.V,LTHM,MIN.AX,PLS.AX,SGML.V,SLI.V,SQM,SYA.AX,WML.V
date,,,,,,,,,,,,,,,,,,,,,
2021-05-28,-0.09,0.02,0.09,0.02,0.07,0.05,-0.05,-0.12,0.09,0.01,...,0.00,0.00,0.07,0.06,0.13,0.05,-0.03,0.03,0.32,0.23
2021-06-04,-0.06,0.08,0.03,0.14,0.05,0.04,0.09,0.01,0.14,-0.01,...,-0.05,0.00,0.06,0.04,0.08,0.11,-0.01,0.06,0.29,-0.26
2021-06-11,0.03,-0.04,-0.06,-0.10,-0.02,-0.02,-0.02,-0.08,0.07,-0.05,...,0.10,0.00,-0.08,0.03,0.07,-0.03,0.01,0.05,0.11,0.06
2021-06-18,-0.05,-0.03,-0.04,-0.06,-0.13,-0.07,-0.07,-0.09,-0.16,-0.11,...,-0.15,0.00,-0.09,0.02,-0.02,-0.05,0.23,-0.12,-0.02,0.03
2021-06-25,-0.05,0.04,0.19,0.03,0.07,0.05,-0.04,0.01,-0.11,0.07,...,0.00,0.86,0.04,0.00,0.06,0.09,0.02,0.11,0.00,-0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-07,-0.03,-0.00,-0.02,-0.01,0.00,0.06,0.00,-0.01,0.06,-0.00,...,0.13,-0.09,-0.00,-0.06,-0.00,-0.00,-0.05,0.09,0.03,0.05
2023-07-14,0.10,0.02,0.02,0.01,0.03,-0.01,-0.07,0.01,-0.03,0.05,...,0.01,0.11,0.03,0.07,0.04,-0.02,0.06,-0.00,0.05,0.03
2023-07-21,-0.09,-0.05,-0.06,-0.03,-0.03,-0.09,0.05,0.00,0.02,-0.00,...,-0.11,-0.08,-0.07,-0.03,-0.04,-0.07,-0.01,-0.05,-0.11,0.02


In [112]:
merge_df = log_return_stocks.merge(target_df, left_index=True, right_index=True).dropna()
merge_df.columns = ['002176.SZ', '002460.SZ', '002466.SZ', '002497.SZ', 'AKE.AX', 'ALB',
       'ALL.L', 'AMLI', 'CRE.V', 'ERA.PA', 'INF.AX', 'LAC', 'LEL.AX', 'LIS.V',
       'LITH.V', 'LPI.AX', 'LTH.V', 'LTHM', 'MIN.AX', 'PLS.AX', 'SGML.V',
       'SLI.V', 'SQM', 'SYA.AX', 'WML.V', 'Price', 'Lithium']
merge_df = merge_df.drop('Price', axis = 1)
merge_df

,002176.SZ,002460.SZ,002466.SZ,002497.SZ,AKE.AX,ALB,ALL.L,AMLI,CRE.V,ERA.PA,...,LTH.V,LTHM,MIN.AX,PLS.AX,SGML.V,SLI.V,SQM,SYA.AX,WML.V,Lithium
2021-05-28,-0.09,0.02,0.09,0.02,0.07,0.05,-0.05,-0.12,0.09,0.01,...,0.00,0.07,0.06,0.13,0.05,-0.03,0.03,0.32,0.23,0.02
2021-06-04,-0.06,0.08,0.03,0.14,0.05,0.04,0.09,0.01,0.14,-0.01,...,0.00,0.06,0.04,0.08,0.11,-0.01,0.06,0.29,-0.26,0.01
2021-06-11,0.03,-0.04,-0.06,-0.10,-0.02,-0.02,-0.02,-0.08,0.07,-0.05,...,0.00,-0.08,0.03,0.07,-0.03,0.01,0.05,0.11,0.06,0.00
2021-06-18,-0.05,-0.03,-0.04,-0.06,-0.13,-0.07,-0.07,-0.09,-0.16,-0.11,...,0.00,-0.09,0.02,-0.02,-0.05,0.23,-0.12,-0.02,0.03,0.01
2021-06-25,-0.05,0.04,0.19,0.03,0.07,0.05,-0.04,0.01,-0.11,0.07,...,0.86,0.04,0.00,0.06,0.09,0.02,0.11,0.00,-0.11,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,-0.12,-0.03,-0.11,-0.08,0.04,0.07,-0.01,-0.02,0.03,-0.06,...,0.23,0.06,-0.07,0.05,-0.02,-0.03,0.06,0.00,-0.06,-0.09
2023-05-05,-0.01,-0.02,0.01,-0.03,0.02,-0.03,-0.08,-0.01,-0.07,-0.01,...,0.02,0.06,-0.05,0.04,0.04,-0.06,0.00,0.00,-0.10,0.00
2023-05-12,-0.00,0.01,0.03,0.00,0.20,0.09,0.08,0.01,0.02,-0.01,...,-0.00,0.10,0.05,0.08,0.08,0.13,0.10,0.07,0.06,0.13
2023-05-19,0.04,0.04,0.06,0.04,0.03,0.04,0.00,-0.04,-0.08,0.05,...,0.02,-0.02,0.03,0.04,0.02,-0.04,-0.01,0.07,0.25,0.13


In [113]:
stock_tickers = list(merge_df.columns)
stock_tickers.pop()


'Lithium'

In [114]:
stock_tickers

['002176.SZ',
 '002460.SZ',
 '002466.SZ',
 '002497.SZ',
 'AKE.AX',
 'ALB',
 'ALL.L',
 'AMLI',
 'CRE.V',
 'ERA.PA',
 'INF.AX',
 'LAC',
 'LEL.AX',
 'LIS.V',
 'LITH.V',
 'LPI.AX',
 'LTH.V',
 'LTHM',
 'MIN.AX',
 'PLS.AX',
 'SGML.V',
 'SLI.V',
 'SQM',
 'SYA.AX',
 'WML.V']

In [115]:
lithium = np.array(merge_df.Lithium)
stocks = np.array(merge_df.drop('Lithium', axis = 1))

In [222]:
N = len(upstream_companies)
model_dic = {}
def objective(params, returns, rlit, j, include_lag, model_results = False):
    weights = params
    
    if j<=0:
        # Calculate the weighted sum of returns
        weighted_returns = np.dot(returns, weights)
        shifted_returns = weighted_returns
    
    else:
        test_df = pd.DataFrame(returns)
        shift_df = test_df.rolling(window = j).mean().shift(1).fillna(0)
        shfit_np = np.array(shift_df)
        shifted_returns = np.dot(shfit_np, weights)
    
    # If rlit(t-1) is to be added as a regressor
    if include_lag:
        #gamma = params[N+2]
        rlit_lag = np.roll(rlit, 1)
        rlit_lag[0] = 0
        
        model_df = pd.DataFrame({'rlit':rlit, 'shifted_returns': shifted_returns, 'r_lit_lag':rlit_lag})
        model = smf.ols('rlit~ shifted_returns + r_lit_lag', data=model_df).fit(cov_type='HAC', cov_kwds={'maxlags': 3})
        
        residuals = model.resid
        #residuals = rlit - (alpha + beta * shifted_returns + gamma * rlit_lag)
    else:
        
        model_df = pd.DataFrame({'rlit':rlit, 'shifted_returns': shifted_returns})
        model = smf.ols('rlit~ shifted_returns', data=model_df).fit(cov_type='HAC', cov_kwds={'maxlags': 3})
        residuals = model.resid 
        #residuals = rlit - (alpha + beta * shifted_returns)
        
    if model_results:
        return model
        
    return np.sum(residuals**2)/len(residuals)*100

cons = (
    {'type': 'eq', 'fun': lambda params: np.sum(params[:N]) - 1},  # Sum of weights = 1
    {'type': 'ineq', 'fun': lambda params: params[:N]}  # Weights >= 0
)

# Initial guess for weights, alpha, and beta
initial_params = np.ones(N) / N

# Bounds for weights, alpha, and beta
bounds = [(0, 1) for _ in range(N)]

# If including lag, add bounds for gamma
bounds_lag = bounds

returns = stocks

rlit = lithium

In [117]:
# without lag: 
results = {"without_lithium(-1)":{},
          "with_lithium(-1)":{}}

errors = {"Null":sum(np.power(lithium - np.mean(lithium), 2))/len(lithium)*100}

for j in range(0,5):
    result_j = minimize(objective, initial_params, args=(returns, rlit, j, False), constraints=cons, bounds=bounds)
    results["without_lithium(-1)"][j] = result_j
    errors[f"Asset(-{j})"]= result_j.fun
    
for j in range(0,5):
    result_j = minimize(objective, initial_params, args=(returns, rlit, j, True), constraints=cons, bounds=bounds)
    results["with_lithium(-1)"][j] = result_j
    errors[f"Lithium(-1)+Asset(-{j})"]= result_j.fun

In [118]:
model_res = []
for j in range(0,5):
    weights = results["without_lithium(-1)"][j].x
    model = objective(params = weights, returns= stocks, rlit = lithium, j = j, include_lag = False, model_results = True)
    
    res = {
        'Asset_Lag': j,
        'intercept':model.params['Intercept'],
    'intercept_pvalue' : model.pvalues['Intercept'],
    'Asset_Lag_coefficient' : model.params['shifted_returns'],
    'Asset_Lag_coefficient_pvalue' : model.pvalues['shifted_returns'],
    'Asset_lag_hac_standard_error' : model.bse['shifted_returns']}
    
    model_res.append(res)

In [119]:
pd.DataFrame(model_res)

,Asset_Lag,intercept,intercept_pvalue,Asset_Lag_coefficient,Asset_Lag_coefficient_pvalue,Asset_lag_hac_standard_error
0,0,0.01,0.29,0.37,0.00,0.13
1,1,0.01,0.34,0.23,0.00,0.08
2,2,0.01,0.47,0.43,0.02,0.18
3,3,0.00,0.60,0.63,0.01,0.24
4,4,0.00,0.73,0.89,0.01,0.35


In [120]:
model_res_df = pd.DataFrame(model_res)
model_res_df.to_csv('Task3.1_without_lithium-1.csv')

In [121]:
stock_weights = []
for j in range(0,5):
    weights = results["without_lithium(-1)"][j].x
    stock_weights.append(weights)
stock_weights = pd.DataFrame(stock_weights)
stock_weights.columns = stock_tickers

In [122]:
stock_weights.to_csv('Task3.1(weights)_without_lithium-1.csv')

In [123]:
model_res2 = []
for j in range(0,5):
    weights = results["with_lithium(-1)"][j].x
    model = objective(params = weights, returns= stocks, rlit = lithium, j = j, include_lag = True, model_results = True)
    
    res = {
    'Asset_Lag': j,
    'intercept':model.params['Intercept'],
    'intercept_pvalue' : model.pvalues['Intercept'],
    'Asset_Lag_coefficient' : model.params['shifted_returns'],
    'Asset_Lag_coefficient_pvalue' : model.pvalues['shifted_returns'],
    'Asset_lag_hac_standard_error' : model.bse['shifted_returns'],
    'lithium_lag1_coefficient' : model.params['r_lit_lag'],
    'lithium_lag1_coefficient_pvalue' : model.pvalues['r_lit_lag'],
    'lithium_lag1_hac_standard_error' : model.bse['r_lit_lag']}
    
    model_res2.append(res)

In [124]:
model_res_df2 = pd.DataFrame(model_res2)
model_res_df2.to_csv('Task3.2_with_lithium-1.csv')

In [125]:
stock_weights2 = []
for j in range(0,5):
    weights = results["with_lithium(-1)"][j].x
    stock_weights2.append(weights)
stock_weights2 = pd.DataFrame(stock_weights2)
stock_weights2.columns = stock_tickers

In [126]:
stock_weights2.to_csv('Task3.2(weights)_with_lithium-1.csv')

# Market Weighted 

In [3]:
import pandas as pd 
import numpy as np 

outstanding = pd.read_csv("outstandingShares.csv")

In [10]:
original_columns = outstanding.columns
converted_columns = ['year']
for col in original_columns[1:]:
    if col.endswith('.AU'):
        converted_columns.append(col.replace('.AU', '.AX'))
    elif col.endswith('.LSE'):
        converted_columns.append(col.replace('.LSE', '.L'))
    elif col.endswith('.SHE'):
        converted_columns.append(col.replace('.SHE', '.SZ'))
    else:
        converted_columns.append(col)

print(converted_columns, len(converted_columns), len(original_columns))


['year', 'ALB', 'AKE.AX', 'AMLI', 'AMG.AS', 'LIT.V', 'ALL.L', 'BM8.AX', '002240.SZ', 'CTL.L', 'CMP', 'CXO.AX', 'CRE.V', 'CRR.AX', 'ERA.PA', 'ESS.AX', 'GLN.AX', '002460.SZ', 'GT1.AX', 'ILI.V', 'INF.AX', 'ILC.V', '002176.SZ', 'KZR.AX', 'LKE.AX', 'LRS.AX', 'LAC', 'LITH.V', 'LEXI.V', 'LEL.AX', 'LTH.V', 'LPI.AX', 'LIS.V', 'LTHM', 'MIN.AX', '1MC.AX', 'NOAL.V', 'PLL.AX', 'PLS.AX', 'PKX', 'PWM.V', 'PE.V', '000792.SZ', 'QMC.V', 'RIO', 'SYA.AX', '002497.SZ', 'SGML', '002738.SZ', 'SQM', 'SLI.V', '002466.SZ', 'ULT.V', 'WML.V', '2899.HK', '002756.SZ', '002192.SZ'] 57 57


In [75]:
outstanding.columns = converted_columns
# get the 2021 market capitalization with outstanding share * stock price? 

import numpy as np
import urllib.request, json

from eod import EodHistoricalData

api = EodHistoricalData("5f3afd582bd7b4.95720069")

def get_ticker_details(ticker):
    url = f"https://eodhistoricaldata.com/api/fundamentals/{ticker}?api_token=5f3afd582bd7b4.95720069&order=d&fmt=json"
    count = 0
    
    try:
        response = urllib.request.urlopen(url, timeout=5)
        data = json.loads(response.read())
    except Exception as e:
        print(f'Failed to retrieve {ticker}: {e}')
        return None
    
    try:
        general_dict = data['General']
        general_df = pd.json_normalize(general_dict)
        general_df = general_df.add_prefix('General.')
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        general_df = None
        count += 1
        
    try:
        highlights_dict = data['Highlights']
        highlights_df = pd.json_normalize(highlights_dict)
        highlights_df = highlights_df.add_prefix('Highlights.')
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        highlights_df = None
        count += 1
    
    try:
        earnings_dict = data['Earnings']
        earnings_df = pd.json_normalize(earnings_dict)
        earnings_df = earnings_df.add_prefix('Earnings.')
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        earnings_df = None
        count += 1
    
    try:
        financials_dict = data['Financials']
        financials_df = pd.json_normalize(financials_dict)
        financials_df = financials_df.add_prefix('Financials.')
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        financials_df = None
        count += 1
    
    if count < 4:
        df = pd.concat([general_df, highlights_df, earnings_df, financials_df], axis=1)
        print(f'Finish fetching {ticker}')
    else:
        return None
        
    return df

dfs = []
tickers = original_columns[1:].copy()
ticker_list = converted_columns[1:]

for i in range(len(tickers)):
    df = get_ticker_details(tickers[i])
    df['ticker'] = ticker_list[i]
    dfs.append(df)

res = pd.concat([df for df in dfs if df is not None])

Finish fetching ALB
Finish fetching AKE.AU
Finish fetching AMLI
Finish fetching AMG.AS
Finish fetching LIT.V
Finish fetching ALL.LSE
Finish fetching BM8.AU
Finish fetching 002240.SHE
Finish fetching CTL.LSE
Finish fetching CMP
Finish fetching CXO.AU
Finish fetching CRE.V
Finish fetching CRR.AU
Finish fetching ERA.PA
Finish fetching ESS.AU
Finish fetching GLN.AU
Finish fetching 002460.SHE
Finish fetching GT1.AU
Finish fetching ILI.V
Finish fetching INF.AU
Finish fetching ILC.V
Finish fetching 002176.SHE
Finish fetching KZR.AU
Finish fetching LKE.AU
Finish fetching LRS.AU
Finish fetching LAC
Finish fetching LITH.V
Finish fetching LEXI.V
Finish fetching LEL.AU
Finish fetching LTH.V
Finish fetching LPI.AU
Finish fetching LIS.V
Finish fetching LTHM
Finish fetching MIN.AU
Finish fetching 1MC.AU
Finish fetching NOAL.V
Finish fetching PLL.AU
Finish fetching PLS.AU
Finish fetching PKX
Finish fetching PWM.V
Finish fetching PE.V
Finish fetching 000792.SHE
Finish fetching QMC.V
Finish fetching RIO

In [79]:
market_info = res[['ticker',
    'General.CurrencyCode',
 'Highlights.MarketCapitalization',
'Highlights.MarketCapitalizationMln']]
market_info.reset_index(drop = True)
market_info.columns = ['ticker', 'currency', 'marketCap', 'marketCapMln']

In [81]:
def get_data(start_date, end_date, ticker):
    if ticker.endswith('.US'):
        ticker = ticker.split('.')[0]
    downloader = YahooDownloader(start_date = start_date, end_date = end_date, ticker_list=[ticker])
    data = downloader.fetch_data()
    df = data[['date', 'close']].copy()
    df.columns = ['Date', 'Price']
    return df

def get_stock_data(ticker, start_date, end_date):
    df = get_data(start_date, end_date, ticker)
    df['Date'] = pd.to_datetime(df['Date'])
    df.columns = ['Date',"Asset_close"]
    df.set_index('Date', inplace=True)
    
    # find the original currency it has 
    from_currency = market_info[market_info.ticker == ticker].currency.iloc[0]
    if from_currency != "CNY":
        ticker_symbol = from_currency + "CNY=X"
        print(ticker_symbol)
        exchange_rate = yf.download(ticker_symbol, start=start_date, end=end_date)
        exchange_rate = exchange_rate[[ "Adj Close"]]
        exchange_rate.columns = ["Exchange_rate"]
        exchange_rate.index = pd.to_datetime(exchange_rate.index)
        df = df.merge(exchange_rate, left_index=True, right_index=True)
        df["converted_close"] = df["Asset_close"]*df["Exchange_rate"]
        df = df[["converted_close"]]
        df.columns = ['Asset_close']
    df['ticker'] = ticker
    
    # Resample to get the last business day's 'Close' price for each week.
    # weekly_df = df['Asset_close'].resample(sampling_period).last()
    # weekly_df = pd.DataFrame(weekly_df)
    # weekly_df['Log_Return'] = np.log(weekly_df["Asset_close"]).diff()

    # The first entry will be NaN because there's no previous data to calculate a return from.
    # So, we remove the first entry
    # weekly_df = weekly_df.dropna()
    return df

def get_list_stock_data(tickers_list, start_date, end_date):
    df = pd.DataFrame(columns = ['Asset_close', 'ticker'])
    for ticker in tickers_list:
        cur_df = get_stock_data(ticker, start_date, end_date)
        df = df.append(cur_df)
    return df

In [82]:
test = get_list_stock_data(tickers_list = ticker_list, start_date = '2017-08-09', end_date = '2023-08-07')

[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1507, 8)
USDCNY=X
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1516, 8)
AUDCNY=X
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1507, 8)
USDCNY=X
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1535, 8)
EURCNY=X
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1503, 8)
CADCNY=X
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (1512, 8)
GBXCNY=X
[

In [87]:
stock_data = test.copy().reset_index()
stock_data.columns = ['date','close', 'ticker']
pivoted_stocks = stock_data.pivot(index='date', columns='ticker', values='close')

In [89]:
pivoted_stocks

ticker,000792.SZ,002176.SZ,002192.SZ,002240.SZ,002460.SZ,002466.SZ,002497.SZ,002738.SZ,002756.SZ,1MC.AX,...,PLS.AX,PWM.V,QMC.V,RIO,SGML,SLI.V,SQM,SYA.AX,ULT.V,WML.V
date,,,,,,,,,,,,,,,,,,,,,
2017-08-09,11.99,11.90,30.72,15.43,29.23,46.34,13.80,11.35,23.50,0.95,...,2.00,1.43,0.63,182.83,NaN,5.39,208.64,0.07,0.90,9.78
2017-08-10,11.36,11.69,30.53,15.75,29.07,45.06,12.89,11.05,23.50,0.92,...,1.94,1.47,0.60,178.68,NaN,5.20,202.20,0.07,0.89,9.60
2017-08-11,11.00,10.99,29.37,14.17,28.27,43.22,12.63,10.47,23.50,0.86,...,1.90,1.51,0.60,175.00,NaN,5.47,201.17,0.07,0.89,9.74
2017-08-14,11.22,11.65,31.12,15.59,31.10,46.70,13.89,10.61,23.50,0.89,...,1.94,1.47,0.52,176.02,NaN,5.88,205.14,0.07,0.84,9.76
2017-08-15,11.16,11.58,30.17,15.82,31.65,47.33,14.68,10.49,23.50,0.89,...,1.90,1.47,0.52,174.03,NaN,5.70,206.82,0.07,0.89,9.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-31,20.00,11.72,61.41,27.52,59.83,66.55,17.82,45.21,54.16,0.03,...,22.13,2.29,0.80,459.56,277.22,32.62,523.41,0.68,0.32,2.13
2023-08-01,19.95,11.64,61.55,27.42,59.29,66.28,17.95,44.89,54.10,0.03,...,22.45,2.31,0.86,452.34,266.37,32.12,511.39,0.74,0.32,2.21
2023-08-02,19.75,11.51,60.69,27.26,58.59,65.41,18.21,43.30,53.83,0.03,...,22.29,2.20,0.81,445.28,263.95,31.67,506.02,0.71,0.38,2.15


In [96]:
pivoted_stocks.index = pd.to_datetime(pivoted_stocks_clean.index)
pivoted_stocks_friday = pivoted_stocks_clean.resample('W-FRI').last()
log_return_stocks = np.log(pivoted_stocks_friday).diff()
log_return_stocks = log_return_stocks.fillna(0) # fill in all of non-avalibale with 0 

In [93]:
log_return_stocks.columns

Index(['000792.SZ', '002176.SZ', '002192.SZ', '002240.SZ', '002460.SZ',
       '002466.SZ', '002497.SZ', '002738.SZ', '002756.SZ', '1MC.AX', '2899.HK',
       'AKE.AX', 'ALB', 'AMG.AS', 'AMLI', 'BM8.AX', 'CMP', 'CRE.V', 'CRR.AX',
       'CXO.AX', 'ERA.PA', 'ESS.AX', 'GLN.AX', 'GT1.AX', 'ILC.V', 'ILI.V',
       'INF.AX', 'KZR.AX', 'LAC', 'LEL.AX', 'LEXI.V', 'LIS.V', 'LIT.V',
       'LITH.V', 'LKE.AX', 'LPI.AX', 'LRS.AX', 'LTH.V', 'LTHM', 'MIN.AX',
       'NOAL.V', 'PE.V', 'PKX', 'PLL.AX', 'PLS.AX', 'PWM.V', 'QMC.V', 'RIO',
       'SGML', 'SLI.V', 'SQM', 'SYA.AX', 'ULT.V', 'WML.V'],
      dtype='object', name='ticker')

In [97]:
merge_df = log_return_stocks.merge(target_df, left_index=True, right_index=True).dropna()
merge_df.columns = ['000792.SZ', '002176.SZ', '002192.SZ', '002240.SZ', '002460.SZ',
       '002466.SZ', '002497.SZ', '002738.SZ', '002756.SZ', '1MC.AX', '2899.HK',
       'AKE.AX', 'ALB', 'AMG.AS', 'AMLI', 'BM8.AX', 'CMP', 'CRE.V', 'CRR.AX',
       'CXO.AX', 'ERA.PA', 'ESS.AX', 'GLN.AX', 'GT1.AX', 'ILC.V', 'ILI.V',
       'INF.AX', 'KZR.AX', 'LAC', 'LEL.AX', 'LEXI.V', 'LIS.V', 'LIT.V',
       'LITH.V', 'LKE.AX', 'LPI.AX', 'LRS.AX', 'LTH.V', 'LTHM', 'MIN.AX',
       'NOAL.V', 'PE.V', 'PKX', 'PLL.AX', 'PLS.AX', 'PWM.V', 'QMC.V', 'RIO',
       'SGML', 'SLI.V', 'SQM', 'SYA.AX', 'ULT.V', 'WML.V', 'Price', 'Lithium']
merge_df = merge_df.drop('Price', axis = 1)
merge_df

,000792.SZ,002176.SZ,002192.SZ,002240.SZ,002460.SZ,002466.SZ,002497.SZ,002738.SZ,002756.SZ,1MC.AX,...,PWM.V,QMC.V,RIO,SGML,SLI.V,SQM,SYA.AX,ULT.V,WML.V,Lithium
2017-09-08,0.09,0.11,0.17,-0.01,0.15,0.07,0.15,-0.05,0.00,0.06,...,-0.06,0.01,-0.04,0.00,0.02,0.00,-0.14,0.07,-0.03,0.13
2017-09-15,0.22,0.05,0.11,0.04,-0.04,-0.04,0.17,0.03,0.00,0.19,...,0.01,0.01,-0.00,0.00,0.29,0.19,0.21,0.15,0.08,0.02
2017-09-22,0.10,-0.08,-0.10,-0.11,-0.07,-0.06,-0.08,0.05,0.00,-0.03,...,0.03,0.46,0.01,0.00,0.07,-0.02,-0.07,-0.10,-0.00,0.01
2017-09-29,0.07,-0.03,0.08,-0.00,0.05,0.02,-0.03,0.06,0.00,0.14,...,0.10,-0.06,-0.00,0.00,-0.01,-0.00,-0.31,0.00,-0.01,0.00
2017-10-20,-0.03,0.01,0.06,0.01,0.05,0.03,0.02,0.00,0.00,-0.03,...,-0.07,0.25,-0.03,0.00,-0.03,-0.01,0.30,0.21,-0.00,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,-0.03,-0.12,-0.05,-0.06,-0.03,-0.11,-0.08,0.02,-0.00,0.10,...,-0.04,-0.00,-0.00,-0.01,-0.03,0.07,-0.01,-0.13,-0.06,-0.09
2023-05-05,-0.01,-0.01,-0.03,-0.03,-0.02,0.01,-0.03,-0.00,-0.02,0.01,...,-0.04,0.00,-0.01,0.05,-0.06,0.00,0.01,0.00,-0.09,0.00
2023-05-12,-0.01,-0.00,-0.03,0.01,0.01,0.03,0.00,0.03,0.05,0.01,...,0.01,-0.04,-0.02,0.07,0.13,0.11,0.08,-0.33,0.07,0.13
2023-05-19,0.01,0.04,0.10,0.08,0.04,0.06,0.04,0.04,0.08,-0.10,...,-0.07,-0.13,0.02,0.04,-0.03,0.00,0.07,0.19,0.26,0.13


In [101]:
stock_tickers = list(merge_df.columns)
stock_tickers.pop()
lithium = np.array(merge_df.Lithium)
stocks = merge_df.drop('Lithium', axis = 1)

In [108]:
market_info.loc[market_info.ticker == 'NOAL.V', 'marketCapMln'] = 36.90

In [124]:
market_info

,ticker,currency,marketCap,marketCapMln
0,ALB,USD,21573072896,"21,573.07"
0,AKE.AX,AUD,9175899858,"9,175.90"
0,AMLI,USD,360620416,360.62
0,AMG.AS,EUR,1009845888,"1,009.85"
0,LIT.V,CAD,27963976,27.96
0,ALL.L,GBX,140158592,140.16
0,BM8.AX,AUD,24677540,24.68
0,002240.SZ,CNY,20638787584,"20,638.79"
0,CTL.L,GBX,61518852,61.52
0,CMP,USD,1190898432,"1,190.90"


In [112]:
currency_list = market_info.currency.unique()

In [139]:
exchange_rate_sumbol = []
for from_currency in currency_list:
    if from_currency not in ['CNY', 'GBX']:
        ticker_symbol = from_currency + "CNY=X"
        exchange_rate_sumbol.append(ticker_symbol)
exchange_rate = yf.download(exchange_rate_sumbol, start='2017-08-09', end='2023-08-09')

[*********************100%***********************]  5 of 5 completed


In [168]:
exchange_rate_close = exchange_rate[[(    'Close', 'AUDCNY=X'),
            (    'Close', 'CADCNY=X'),
            (    'Close', 'EURCNY=X'),
            (    'Close', 'HKDCNY=X'),
            (    'Close', 'USDCNY=X')]]
exchange_rate_close.columns = ['AUD', 'CAD', 'EUR', 'HKD', 'USD']

exchange_rate_mean = pd.DataFrame(exchange_rate_close.mean())
exchange_rate_mean = exchange_rate_mean.reset_index()
exchange_rate_mean.columns = ['currency', 'avg_exchange_rate']
exchange_rate_mean.loc[len(exchange_rate_mean.index)] = ['CNY', 1] 
exchange_rate_mean

,currency,avg_exchange_rate
0,AUD,4.81
1,CAD,5.16
2,EUR,7.62
3,HKD,0.86
4,USD,6.74
5,CNY,1.00


In [183]:
market_info_m = market_info.merge(exchange_rate_mean, left_on = 'currency', right_on = 'currency')
market_info_m['marketCapMln_RMB'] = market_info_m.marketCapMln*market_info_m.avg_exchange_rate

In [185]:
market_info_m['weight'] = market_info_m.marketCapMln_RMB.apply(lambda x: x/sum(market_info_m.marketCapMln_RMB))

In [219]:
weights = []
for col in stocks.columns:
    weights.append(market_info_m[market_info_m.ticker == col].weight.values[0])
weights



In [225]:
stocks = stocks.drop(['weighted_return'], axis = 1)

In [226]:
stocks

,000792.SZ,002176.SZ,002192.SZ,002240.SZ,002460.SZ,002466.SZ,002497.SZ,002738.SZ,002756.SZ,1MC.AX,...,PLS.AX,PWM.V,QMC.V,RIO,SGML,SLI.V,SQM,SYA.AX,ULT.V,WML.V
2017-09-08,0.09,0.11,0.17,-0.01,0.15,0.07,0.15,-0.05,0.00,0.06,...,0.12,-0.06,0.01,-0.04,0.00,0.02,0.00,-0.14,0.07,-0.03
2017-09-15,0.22,0.05,0.11,0.04,-0.04,-0.04,0.17,0.03,0.00,0.19,...,0.25,0.01,0.01,-0.00,0.00,0.29,0.19,0.21,0.15,0.08
2017-09-22,0.10,-0.08,-0.10,-0.11,-0.07,-0.06,-0.08,0.05,0.00,-0.03,...,-0.06,0.03,0.46,0.01,0.00,0.07,-0.02,-0.07,-0.10,-0.00
2017-09-29,0.07,-0.03,0.08,-0.00,0.05,0.02,-0.03,0.06,0.00,0.14,...,0.19,0.10,-0.06,-0.00,0.00,-0.01,-0.00,-0.31,0.00,-0.01
2017-10-20,-0.03,0.01,0.06,0.01,0.05,0.03,0.02,0.00,0.00,-0.03,...,-0.06,-0.07,0.25,-0.03,0.00,-0.03,-0.01,0.30,0.21,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,-0.03,-0.12,-0.05,-0.06,-0.03,-0.11,-0.08,0.02,-0.00,0.10,...,0.04,-0.04,-0.00,-0.00,-0.01,-0.03,0.07,-0.01,-0.13,-0.06
2023-05-05,-0.01,-0.01,-0.03,-0.03,-0.02,0.01,-0.03,-0.00,-0.02,0.01,...,0.04,-0.04,0.00,-0.01,0.05,-0.06,0.00,0.01,0.00,-0.09
2023-05-12,-0.01,-0.00,-0.03,0.01,0.01,0.03,0.00,0.03,0.05,0.01,...,0.09,0.01,-0.04,-0.02,0.07,0.13,0.11,0.08,-0.33,0.07
2023-05-19,0.01,0.04,0.10,0.08,0.04,0.06,0.04,0.04,0.08,-0.10,...,0.04,-0.07,-0.13,0.02,0.04,-0.03,0.00,0.07,0.19,0.26


In [227]:
model_res = []
for j in range(0,5):
    weights = weights
    model = objective(params = weights, returns= stocks, rlit = lithium, j = j, include_lag = False, model_results = True)
    
    res = {
        'Asset_Lag': j,
        'intercept':model.params['Intercept'],
    'intercept_pvalue' : model.pvalues['Intercept'],
    'Asset_Lag_coefficient' : model.params['shifted_returns'],
    'Asset_Lag_coefficient_pvalue' : model.pvalues['shifted_returns'],
    'Asset_lag_hac_standard_error' : model.bse['shifted_returns']}
    
    model_res.append(res)

In [229]:
pd.DataFrame(model_res)

,Asset_Lag,intercept,intercept_pvalue,Asset_Lag_coefficient,Asset_Lag_coefficient_pvalue,Asset_lag_hac_standard_error
0,0,0.00,0.74,0.06,0.14,0.04
1,1,0.00,0.70,0.00,0.97,0.03
2,2,0.00,0.72,0.03,0.65,0.07
3,3,0.00,0.72,0.04,0.71,0.10
4,4,0.00,0.72,0.03,0.82,0.12


In [231]:
model_res2 = []
for j in range(0,5):
    weights = weights
    model = objective(params = weights, returns= stocks, rlit = lithium, j = j, include_lag = True, model_results = True)
    
    res = {
    'Asset_Lag': j,
    'intercept':model.params['Intercept'],
    'intercept_pvalue' : model.pvalues['Intercept'],
    'Asset_Lag_coefficient' : model.params['shifted_returns'],
    'Asset_Lag_coefficient_pvalue' : model.pvalues['shifted_returns'],
    'Asset_lag_hac_standard_error' : model.bse['shifted_returns'],
    'lithium_lag1_coefficient' : model.params['r_lit_lag'],
    'lithium_lag1_coefficient_pvalue' : model.pvalues['r_lit_lag'],
    'lithium_lag1_hac_standard_error' : model.bse['r_lit_lag']}
    
    model_res2.append(res)

In [233]:
pd.DataFrame(model_res2)

,Asset_Lag,intercept,intercept_pvalue,Asset_Lag_coefficient,Asset_Lag_coefficient_pvalue,Asset_lag_hac_standard_error,lithium_lag1_coefficient,lithium_lag1_coefficient_pvalue,lithium_lag1_hac_standard_error
0,0,0.00,0.93,0.07,0.04,0.03,0.50,0.00,0.15
1,1,0.00,0.82,-0.02,0.50,0.03,0.50,0.00,0.15
2,2,0.00,0.85,0.00,0.98,0.05,0.50,0.00,0.15
3,3,0.00,0.84,-0.00,0.95,0.07,0.50,0.00,0.15
4,4,0.00,0.83,-0.02,0.83,0.08,0.50,0.00,0.15
